# Data les 3 - geavanceerdere data exploratie met pandas

Notebook bij les 2 van de leerlijn data van S3 - AI. 

© Auteur: Rianne van Os

**Voorbereiding voor les 3:** Lees de theorie en maak t/m opdracht 3.2 van dit notebook

In de vorige les hebben we een basis gelegd in pandas en gezien hoe je visualisaties kunt gebruiken om inzicht te krijgen in je data. In deze les breiden we die kennis uit met technieken om uitgebreidere statistieken weer te geven, je leert meerdere datasets samenvoegen en je leert omgaan met missende waarde of outliers in je data.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Statistieken per groep

Volg deze tutorial over het berekenen van statistieken per groep, vanaf **aggregating statistices grouped by category**: https://pandas.pydata.org/docs/getting_started/intro_tutorials/06_calculate_statistics.html

Dit groeperen hebben jullie in S2 ook al geleerd in SQL. 

Wat de tutorial niet behandelt, maar wat wel erg handig is, is de combinatie van group_by() en agg(). Hiermee kun je eerst de dataset groeperen, en vervolgens in de .agg() method aangeven hoe je bepaalde kolommen wilt aggregeren. In onderstaande voorbeeld laat ik dat zien, samen met nog wat andere tips voor het werken met gegroepeerde dataframes.

### groupby en agg
We kijken opnieuw naar de dataset vgsales en we zijn geinteresseerd in statistieken van iedere Publisher per jaar. We willen namelijk van iedere publisher weten hoeveel spellen die per jaar uitgebracht heeft en ook hoeveel geld dat ieder jaar opleverde.
Het dataframe met deze statistieken kunnen we in één regel code maken, door gebruik te maken van de combinatie van `groupby` en `agg`. 

Hiervoor groeperen we dat data op `Year` en `Publisher`. Om de opbrengst per publisher per jaar te bepalen berekenen we vervolgens de `sum` van `Global_Sales`. Het totaal aantal verkochte spellen is niks anders dan het aantal rijen in die groep, daarvoor kunnen we een willekeurige kolom nemen en daar het aantal rijen van tellen, met behulp van `count`.

In [ ]:
#dataset inlezen
vgsales = pd.read_csv('../databronnen/vgsales.csv')

#groeperen en aggregeren
vgsales_grouped_publisher_year = vgsales.groupby(['Year', 'Publisher']).agg({'Global_Sales':'sum', 'Name':'count'})

#resultaat tonen
vgsales_grouped_publisher_year.head()

Na een groupby krijg je een beetje een raar dataframe, met een zogenoemde 'multi index', de combinatie van Year en Publisher ziet hij hier als index van de rijen. Dat zie je bij uitvoer van de onderstaande cel:

In [ ]:
vgsales_grouped_publisher_year.index

Om hier weer een 2-dimensionale tabel van te maken zoals je gewend bent, gebruik je de .reset_index() method. 

In [ ]:
vgsales_grouped_publisher_year = vgsales_grouped_publisher_year.reset_index()

Als je onderstaande cel runt zie je dat je weer een dataframe hebt zoals je gewent bent:

In [ ]:
vgsales_grouped_publisher_year.head(10)

Nu moeten we nog de kolomnamen aanpassen, want de namen Global_Sales en Name zijn niet meer toereikend.

In [ ]:
vgsales_grouped_publisher_year.columns = ['Year', 'Publisher', 'Total sales', 'Number of games']
vgsales_grouped_publisher_year.head()

In dit voorbeeld hebben we de som van de sales (`sum`) en het aantal observaties per groep `count` uitgerekend, maar er is nog veel meer mogelijk. Onder andere: 
- mean: Compute mean of groups
- sum: Compute sum of group values
- size: Compute group sizes
- count: Compute count of group - (deze telt NAN's niet mee, waar size dat wel doet)
- std: Standard deviation of groups
- min: Compute min of group values
- max: Compute max of group values

Probeer ze eens uit! 

Als je een berekening per groep wilt doen die niet in de standaardfunctionaliteit zit, kun je zelf een functie hiervoor schrijven en die aanroepen in de `.agg(..)` methode. 

Stel bijvoorbeeld dat het ons is opgevallen dat sommige publishers kiezen voor hele korte en andere voor hele lange spelnamen. Laten we dan eens per publisher uit gaan rekenen wat de gemiddelde lengte van een spelnaam is. Klopt het echt dat sommige veel vaker voor langere namen kiezen?

Hiervoor schrijven we een functie die, gegeven een kolom met spelnamen, de gemiddelde lengte van die namen kan berekenen. 

In [ ]:
#Let op, de paramater die deze functie meekrijgt is van het type Series (dus 1 kolom van het dataframe)
def bepaal_lengte_langste_naam(x: pd.Series) -> int:
    return x.str.len().mean()

Merk op: we hebben hier gebruik gemaakt van `.str`, dat het mogelijk maakt om de waardes in de Series als strings te benaderen, waardoor we functies op deze strings aan kunnen roepen. (Mocht je dit zelf nodig gaan hebben, zie: https://pandas.pydata.org/docs/reference/series.html#api-series-str)

Nu kunnen we deze functie gebruiken in een groupby-aggregatie.

In [ ]:
#groepeer de dataset op publisher en bepaal de gemiddelde lengte van de namen van de spellen
lengte_spelnaam_per_publisher_df = vgsales.groupby('Publisher').agg({'Name': bepaal_lengte_langste_naam}).reset_index()

In [ ]:
#bekijk het dataframe
lengte_spelnaam_per_publisher_df

In [ ]:
#we passen de kolomnamen aan
lengte_spelnaam_per_publisher_df.columns = ['publisher', 'aantal_tekens_in_spelnaam']

In [ ]:

lengte_spelnaam_per_publisher_df.sort_values('aantal_tekens_in_spelnaam', ascending=False)

We zien dat er inderdaad een groot verschil is in de lengte van de namen van de spellen. Er zijn publishers waarbij de gemiddelde spelnaam meer dan 70 tekens bevat, terwijl dit bij anderen maar 4 is. Ik vraag me nu meteen af of die publishers met die extreem lange en korte spelnamen meerdere spellen uitgebracht hebben of dat dit maar om 1 spel gaat. Misschien wil je eigenlijk alleen de Publishers bekijken die meer dan 1 spel uitgebracht hebben? Dit mag je zelf onderzoeken als je wilt!

### Data opdracht 3.1
In deze oefeningen werken we weer met de vgsales dataset.

1. Maak een dataframe waarin per publisher te zien is hoeveel spellen hij in totaal uitgegeven heeft. Geef de kolommen duidelijke namen en sorteer de data van meeste naar minste aantal spellen
2. Maak een dataset waarin per genre te zien is wat de totale opbrengst voor dat genre is. Geef de kolommen duidelijke namen en sorteer de data van grootste opbrengst naar kleinste.
3. Welke genre leverde in 2009 het meeste op in Japan? En in europa?

## Datasets combineren

Volg deze tutorial: https://pandas.pydata.org/docs/getting_started/intro_tutorials/08_combine_dataframes.html

Hierin wordt het mergen (oftewel joinen) van dataframes behandelt. Dit hebben jullie ook al in S2 gedaan in SQL. Wat hier niet goed toegelicht wordt het is verschil tussen verschillende typen joins die je uit kunt voeren. Hier een korte uitleg:

- *inner* join: alleen de rijen waarbij de key in beide dataframes voorkomt worden meegenomen
- *left* join: het linker dataframe wordt volledig meegenomen, als er geen match is bij de key in het linker dataframe, dan worden de waardes van het rechter dataframe op die plek NaN
- *right* join: het rechter dataframe wordt volledig meegenomen, als er geen match is bij de key in het rechter dataframe, dan worden de waardes van het linker dataframe op die plek NaN
- *outer* join: alle rijen van beide dataframes worden meegenomen, als er geen match is bij de key in het linker dataframe, dan worden de waardes van het rechter dataframe op die plek NaN en vice versa

Merk op dat een *right* join en een *left* join dus hetzelfde zijn als je de volgorde van de dataframes omdraait.
Deze venn-diagrammen tonen nog eens het verschil tussen de join-types:


![Verschillende type joins](../afbeeldingen/Data/join_types.png "Verschillende type joins")
bron: https://www.geeksforgeeks.org/different-types-of-joins-in-pandas/

### Data opdracht 3.2
Om te oefenen met het mergen van dataframes gaan we opnieuw naar de datasets met cijfers kijken die we ook al in de vorige 2 lessen gebruikt hebben. 
1. Lees de datasets blokAcijfers.csv en blokAstudenten.csv in als pandas datafame. Geef de dataframes beschrijvende namen en bekijk of de datasets goed zijn ingelezen.
2. Voeg de 2 datasets samen door een join uit te voeren, gebruik hiervoor het studentnummer als *key*. Voer eerst een inner join uit zorg dat je een dataset krijgt met de kolommen `Studentnr`; `Naam`; `Vooropleiding`; `RAD`; `ANA`; en `CBA`.
3. Voer nu een *left join*, een *right join* een *inner join* en een *outer join* uit. Verklaar waarom dit verschillende datasets oplevert.
4. Ga verder met de dataset die je gekregen hebt met de inner join. Controleer of de vooropleiding een rol speelt bij de cijfers die voor een vak gehaald worden. Groepeer hiervoor het dataframe op vooropleiding en bereken het gemiddelde cijfer per vak.

## Data opschonen

Als je in de praktijk met een dataset aan de slag gaat, zul je zien dat de data vaak niet meteen bruikbaar is. Dit kan komen doordat er data ontbreekt, doordat er inconsistenties zijn in de data (bijvoorbeeld verschillende schrijfwijzen voor hetzelfde), of doordat er waarden in je dataset staan die overduidelijk niet kunnen kloppen. We noemen de data waar je mee start de *ruwe data* (of: *raw data*). Voordat je analyses kunt uitvoeren, visualisaties kunt maken of machine learning modellen kunt trainen, zul je die ruwe data moeten opschonen (*cleanen*). Dit is vaak een van de meest tijdrovende stappen in data science, maar ook een van de belangrijkste.

Twee belangrijke onderdelen van data cleaning zijn het omgaan met *missing values* en *outliers*.

### Het vinden van missing values

Vaak heb je in echte datasets te maken met missende waarden die je analyses of je visualisaties kunnen beinvloeden. Je zult dus iets met die data moeten doen. Hierbij is het belangrijk om je af te vragen waarom de data mist en of het missen zelf je al informatie geeft. Maar hiervoor zul je eerst moeten kijken welke data mist en hoeveel. Daar zijn in python verschillende manieren voor, bijvoorbeeld:


In python kun je de missende waarden in je dataset als volgt vinden:
- `df.info()` geeft een overzicht van de datatypes en het aantal niet-missende waarden per kolom
- `df.isnull().sum()` geeft het aantal missende waarden per kolom
- `df[df.isnull().any(axis=1)]` geeft de rijen met missende waarden weer.

### Data opdracht 3.3
In de vorige les heb je gewerkt met airbnb data. Je hebt toen als het goed is de dataset `listings.csv` bekeken van een door jou gekozen stad. Deze dataset is een opgeschoonde versie van de dataset `listings.csv.gz`. Download die laatste dataset, lees hem in als pandas dataframe en ga in de dataset op zoek naar de missende waarde. 

### Omgaan met missing values

Als je de missende waarden gevonden hebt, dan kun je jezelf de volgende vragen stellen:
1. Waarom ontbreekt de data?
2. Is er een manier om deze data alsnog te verkrijgen?
3. Zo niet, wat is dan de beste manier om met de ontbrekende data om te gaan?
    - De rijen met missende waarden uit de dataset verwijderen.
    - De kolommen met missende waarden uit de dataset verwijderen.
    - De ontbrekende waarden vervangen door een andere waarde (bijvoorbeeld 0 of een centrummaat van die feature).
    - De informatie in je analyse gebruiken, bijvoorbeeld door apart het aantal missende waarden te vermelden.

Het antwoord op deze vragen hang sterk af van de context. Hierbij is het van belang waar de data vandaan komt, maar ook wat je met de data wilt gaan doen. Een voorbeeld:

Stel dat er in de dataset een feature is voor inkomen, maar dat er bij een aantal observaties geen inkomen bekend is. Dit kan betekenen dat deze mensen geen inkomen hebben, maar het kan ook zijn dat ze het inkomen niet willen delen. Als de data mist omdat een persoon geen inkomen heeft, dan kan dat juist interessante informatie zijn voor je analyse en kun je de missende waarden vervangen met 0. Mist de waarde omdat de persoon het inkomen niet wilde delen, dan is het lastiger om te bepalen wat je ermee moet. Je kunt dan die rijen verwijderen, of ze vervangen door het gemiddelde inkomen.  Ook kun je dit aantal apart rapporteren, misschien is dat juist wel interessante informatie. 

Het kan natuurlijk ook dat data mist door een technische fout en soms is helemaal niet te achterhalen waarom data mist.



#### Missende waarden verwijderen
De meest simpele methode is het verwijderen van rijen of kolommen die missende waarden bevatten. Hiermee is je dataset in 1 keer opgeschoond, maar je gooit natuurlijk ook veel informatie weg, wat vaak niet wenselijk is. Kies je er toch voor om dat te doen, dan kan dat in pandas met de functie `dropna()`.

- `df.dropna()`: Verwijder alle rijen die *ten minste één* missende waarde bevatten.
- `df.dropna(axis=1)`: Verwijder alle kolommen die *ten minste één* missende waarde bevatten.



#### Missende waarden vullen (imputation)

In plaats van data te verwijderen, kun je missende waarden ook invullen met een andere waarde. Dit proces heet *imputatie*. Pandas heeft hiervoor de functie `fillna()`, zie: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html

Welke waarde je kiest, hangt af van de context:

- Invullen met een constante waarde (bv. 0, 'Onbekend'):  Geschikt als de waarde een logische betekenis heeft, bijvoorbeeld 0 als iemand geen inkomen heeft, of als je het aantal rijen met missende waarden in je analyse mee wilt nemen.
- Invullen met een centrummaat (mean, median, mode). Je kunt hierbij ook de mean, median of mode per groep gebruiken. Dus bijvoorbeeld het gemiddelde inkomen per geslacht of leeftijdscategorie gebruiken.
- Invullen met aangrenzende waarden (forward fill, backward fill): Geschikt voor tijdreeksdata, waarbij je aanneemt dat een missende waarde waarschijnlijk hetzelfde is als de vorige (`method = ffill`) of volgende (`method = bfill`) waarde.
- Invullen met een voorspelling: Een machine learning model gebruiken om de missende waarden te voorspellen - dit is nu nog out of scope maar na volgende week kun je dit.

### Data opdracht 3.4
In de vorige opdracht heb je de missende waarden in de airbnb dataset bekeken. 
- Bedenk samen met een mede-student voor iedere kolom met missende waarden een reden waarom de data kan missen.
- Kies 3 kolommen met missende waarden uit en pas een logische strategie toe om de missende waarden weg te werken. **Merk op:** in de praktijk is de keuze van hoe je met de missende waarden omgaat afhankelijk van wat je met de data wilt doen. Misschien wil je voor verschillende visualisaties wel verschillende technieken toepassen. In deze opdracht laten we die context even weg en moet je alleen oefenen met de techniek.

## Outliers

Naast missende waarden kunnen ook outliers je analyses of visualisaties vertroebelen. Outliers zijn datapunten die ver af liggen van andere data. Dit kan komen door een fout in de data, maar het kan ook gewoon een correct datapunt zijn. Denk bijvoorbeeld aan een dataset met huizenprijzen, als je daar een huis tussen hebt staan dat 10 miljoen euro kost, dan is dat een outlier. Dit kan komen omdat het huis echt 10 miljoen euro kost, maar het kan ook zijn dat er een 0 teveel is ingevoerd. Een ander voorbeeld is een dataset met persoonskenmerken, als daar een lengte van meer dan 3 meter in staat, weet je dat het een fout is. Staat er dan weer een lengte van 173 meter, dan kan het waarschijnlijk zijn dat dit 1.73 meter had moeten zijn.

Het is cruciaal om, net als bij missende waarden, te proberen te achterhalen *waarom* een outlier er is. Een outlier die een fout is, wil je meestal corrigeren of verwijderen. Een outlier die een geldige, maar extreme, observatie is, moet je zorgvuldig behandelen: soms moet je die juist meenemen omdat extreme gevallen belangrijk zijn, soms moet je er toch rekening mee houden dat die ene waarde je visualisatie verstoort of sterk van invloed is op berekende statistieken.

Er zijn verschillende manieren om outliers te vinden:

1. Door je data te visualiseren met een boxplot, histogram of scatterplot.
2. Met behulp van statistieken zoals de z-score of de IQR-methode (hier gaan we nu niet op in)

### Outliers vinden met visualisaties

Als voorbeeld bekijken we de Global_Sales kolom van de vgsales dataset.

In [ ]:
vgsales.boxplot(column='Global_Sales')

Dit is direct een mooi voorbeeld van hoe outliers de visualisatie vertroebelen. Door een hele hoge sales, namelijk een die hoger is dan 80, kunnen we de boxplot zelf niet eens meer goed zien. 

Laten we de dataset filteren op Global_Sales die kleiner zijn dan 80:

In [ ]:
vgsales.loc[vgsales['Global_Sales'] < 80].boxplot(column='Global_Sales')

Je ziet nog steeds dat er enorm veel outliers in de `Global_Sales` kolom zijn. Dit komt omdat de meeste spellen relatief weinig verkopen, maar een klein aantal spellen extreem veel verkoopt. Dit zijn waarschijnlijk *geen* fouten, maar valide, extreme datapunten. Misschien zijn dit eigenlijk juist de meest interessante datapunten in deze dataset. Je kunt er ook voor kiezen juist deze grote waarden te laten zien:

In [ ]:
vgsales.loc[vgsales['Global_Sales'] >5].boxplot(column='Global_Sales')

Naast boxplots kunnen ook histogrammen je helpen om outliers te vinden. Dit mag je zelf toepassen in onderstaande opdracht.

### Outliers vinden met een scatterplot
Soms is een punt als je enkel naar 1 feature kijkt geen outlier, maar is een combinatie van features wil extreem. Hiervoor bekijken we het volgende simpele datasetje met lengtes en gewichten van personen.

In [ ]:
personen = pd.DataFrame({'lengte': [1.60, 1.678, 1.755, 1.764, 1.821, 1.809, 1.851, 1.848, 1.902], 
                         'gewicht': [95, 62.2, 67.2, 72.7, 76.4, 75.2, 81.3, 87.4, 90.4]})

Als we van iedere feature los een boxplot maken zien we niks raars:

In [ ]:
personen.boxplot(column='lengte')

In [ ]:
personen.boxplot(column='gewicht')

Maar maken we een scatterplot, dan zie je wel een outlier: 

In [ ]:
sns.scatterplot(data = personen, x='lengte', y='gewicht')

In deze scatterplot zien we heel duidelijk dat alle punten ongeveer op een lijn liggen, terwijl een 1 specifiek punt daar niet aan voldoet. Ook dit kun je als een outlier beschouwen. Ook hier kan het prima een valide punt zijn, misschien zelfs wel weer het interessantste punt, maar dat hangt weer helemaal af van wat je met je visualisatie wilt laten zien.

### Omgaan met outliers in je data
Als je outliers gevonden hebt, moet je een bedenken wat je ermee wilt gaan doen. We onderscheiden 2 gevallen:
1. Het is een foutieve waarde: in dit geval kun je het beschouwen als een missende waarde en een van de hierboven beschreven technieken toepassen.
2. Het is een valide waarde, maar extreme waarde, dan moet je je bedenken of je het punt wel of niet mee wilt nemen in je visualisatie. Dit is afhankelijk van wat je wilt vertellen met je visualisatie of analyse.

### Data opdracht 3.5
Maak boxplots of histogrammen van de numerieke kolommen van airbnb dataset van jouw stad naar keuze. Kun je outliers vinden in deze kolommen? Denk je dat dit fouten zijn of zijn het valide, maar extreme waarden?

## Data opdracht 3.6 - voorbereiding op PI

In de vorige les heb je een stad gekozen waarvan je de airbnb-data bent gaan analyseren. Hier gaan we nu mee door, alleen pakken we nu de ruwe airbnb data. Ga weer naar https://insideairbnb.com/get-the-data/, kies dezelfde (of een andere) stad en download listings.csv.gz.

Deze data is minder 'clean' dan de dataset die je gebruikte in de vorige les. 

1. Ga op zoek naar missende waardes en outliers.  Als je outliers vindt, bedenk dan: is dit foutieve data of is het te verklaren dat deze waarden voorkomen? Kun je de null-waardes verklaren? 
Wil je deze outliers en null-waardes wel of niet uit je dataset verwijderen bij het maken van visualisaties of het doen van analyses? Probeer dit te beargumenteren. En misschien hangt is dit voor iedere visualisatie die je wilt maken wel anders. Maak een opgeschoonde versie van de dataset die je in de volgende stap gaat gebruiken voor de visualisaties. 

2. Nu ga je de eerste stappen zetten om een dashboard te maken om klanten van airbnb een goed aanbod te laten zien van wat er in de door jou gekozen stad te huur is. Het is natuurlijk leuk een kaartje te laten zien met daarop de locaties van de apparatementen, maar toon bijvoorbeeld ook een overzicht van de gemiddelde prijs per buurt, of de grootte van het aanbod per type appartement. 
Bedenk steeds eerst wat je wilt laten zien en waarom, leg dit voor aan een docent of mede-student en ga dan pas de visualisatie maken. Nu maak je deze nog in een notebook, in de volgende les gaan we hier een dashboard van maken. Bij het maken van de visualisaties mag je gebruik maken van een LLM, je kunt deze vragen om gebruik te maken van het *seaborn* package. Om enigszins de code te begrijpen en aanpassingen te maken kun je de door de introdution van seaborn kijken (https://seaborn.pydata.org/tutorial/introduction.html).

3. (Extra:) Breid je dataset uit door deze te mergen met reviews.csv of calendar.csv. Bedenk opnieuw welke inzichten je met deze data op kunt doen en laat dit zien in een passende visualisatie. Als je met de reviews aan de slag gaat, zul je mogelijk met strings moeten gaan werken, zie hier hoe dat moet: https://pandas.pydata.org/docs/getting_started/intro_tutorials/10_text_data.html.
Ga je met calendar.csv aan de slag, dan kom je datetimes tegen, daar helpt dit bij: https://pandas.pydata.org/docs/getting_started/intro_tutorials/09_timeseries.html